In [1]:
import pandas as pd

In [2]:
reservaciones = pd.read_excel("../data/ia_reservaciones_expandida_limpia.xlsx")
reservaciones

,ID_Reserva,ID_Paquete,ID_Segmento_Comp,ID_Agencia,ID_Tipo_Habitacion,ID_canal,ID_estatus_reservaciones,h_res_fec_ok,h_fec_lld_ok,h_fec_sda_ok,h_tot_hab,h_num_per,h_num_noc,fecha_ocupacion
0,0,1,14,112,Otro,10,9,2019-08-16,2019-11-07,2019-11-10,1.0,2,3.0,2019-11-07
1,0,1,14,112,Otro,10,9,2019-08-16,2019-11-07,2019-11-10,1.0,2,3.0,2019-11-08
2,0,1,14,112,Otro,10,9,2019-08-16,2019-11-07,2019-11-10,1.0,2,3.0,2019-11-09
3,1,2,14,112,Otro,10,9,2019-10-22,2019-12-01,2019-12-05,1.0,4,4.0,2019-12-01
4,1,2,14,112,Otro,10,9,2019-10-22,2019-12-01,2019-12-05,1.0,4,4.0,2019-12-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673249,203000,1,5,Otro,Otro,1,9,2019-08-10,2019-08-10,2019-08-12,0.0,0,0.0,2019-08-11
673250,203001,2,14,112,Otro,1,9,2019-02-24,2019-02-24,2019-02-28,0.0,0,0.0,2019-02-24
673251,203001,2,14,112,Otro,1,9,2019-02-24,2019-02-24,2019-02-28,0.0,0,0.0,2019-02-25
673252,203001,2,14,112,Otro,1,9,2019-02-24,2019-02-24,2019-02-28,0.0,0,0.0,2019-02-26
